In [1]:
library(Seurat)
library(Matrix)
library(reticulate)
library(glmpca)
library(peakRAM)
library(scry)
library(SingleCellExperiment)
library(glmGamPoi)
library(harmony)

Warning message in fun(libname, pkgname):
“rgeos: versions of GEOS runtime 3.9.1-CAPI-1.14.2
and GEOS at installation 3.9.1dev-CAPI-1.14.1differ”
Attaching SeuratObject

Attaching sp

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods

In [2]:
de.facLocs <- c(1.2, 1.5, 2, 2.5, 3, 4)

for (de.facLoc in de.facLocs){
    mtx <- readMM(paste0('/data01/hanbin973/fastrna_r/simdata/', '5_2_2000_', de.facLoc, '.mtx'))
    meta <- read.csv(paste0('/data01/hanbin973/fastrna_r/simdata/', 'meta_5_2_2000_', de.facLoc, '.csv'), row.names=1)
    gene <- read.csv(paste0('/data01/hanbin973/fastrna_r/simdata/', 'gene_5_2_2000_', de.facLoc, '.csv'), row.names=1)

    rownames(mtx) <- 1:nrow(mtx)
    colnames(mtx) <- rownames(meta)

    covmat <- model.matrix(as.formula('~factor(Batch)'), meta)
    covmat <- covmat[,2:ncol(covmat),drop=FALSE]
    meta[,paste0('batch_',1:ncol(covmat))] <- covmat
    
    obj <- CreateSeuratObject(mtx, meta.data=meta)
    var_feature <- rep(0,nrow(obj))
    names(var_feature) <- 1:nrow(obj)

    obj <- SCTransform(obj, variable.features.n=2000, 
                           latent_var=c('log_umi', paste0('batch_',1:ncol(covmat))), 
                           method="glmGamPoi",
                          verbose=FALSE)
    var_feature[rownames(HVFInfo(obj))] <- HVFInfo(obj, 'sct')$residual_variance
    write.csv(var_feature, paste0('feature_var/simdata/', 'var_sct_5_2_2000_', de.facLoc, '.csv'))
    
    obj <- SingleCellExperiment(list(counts=mtx))
    obj <- devianceFeatureSelection(obj, assay="counts", sorted=FALSE, batch=as.factor(meta$Batch))
    feature_var <- rowData(obj)$binomial_deviance
    write.csv(feature_var, paste0('feature_var/simdata/', 'var_scry_5_2_2000_', de.facLoc, '.csv'))
    
    obj <- CreateSeuratObject(mtx, meta.data=meta)
    obj <- NormalizeData(obj, verbose=FALSE)
    obj <- FindVariableFeatures(obj, verbose=FALSE, nfeatures=2000)
    write.csv(HVFInfo(obj)$variance.standardized, paste0('feature_var/simdata/', 'var_log_5_2_2000_', de.facLoc, '.csv'))
    
}

In [3]:
obj <- CreateSeuratObject(mtx, meta.data=meta)
var_feature <- rep(0,nrow(obj))
names(var_feature) <- 1:nrow(obj)

obj <- SCTransform(obj, variable.features.n=2000, 
                       latent_var=c('log_umi', paste0('batch_',1:ncol(covmat))), 
                       method="glmGamPoi",
                      verbose=FALSE)
var_feature[rownames(HVFInfo(obj))] <- HVFInfo(obj, 'sct')$residual_variance
write.csv(var_feature, paste0('feature_var/simdata/', 'var_sct_5_2_2000_', de.prob, '.csv'))

In [4]:
obj <- SingleCellExperiment(list(counts=mtx))
obj <- devianceFeatureSelection(obj, assay="counts", sorted=FALSE)
feature_var <- rowData(obj)$binomial_deviance
write.csv(feature_var, paste0('feature_var/simdata/', 'var_scry_5_2_2000_', de.prob, '.csv'))

In [5]:
obj <- CreateSeuratObject(mtx, meta.data=meta)
obj <- NormalizeData(obj, verbose=FALSE)
obj <- FindVariableFeatures(obj, verbose=FALSE, nfeatures=2000)
write.csv(HVFInfo(obj)$variance.standardized, paste0('feature_var/simdata/', 'var_log_5_2_2000_', de.prob, '.csv'))